In [ ]:
"""
코랩용 DPO (Direct Preference Optimization) 학습 스크립트
./finetuning_data_dpo의 cycle_01.csv 파일을 토대로 1 사이클 DPO 학습 이후
./checkpoints_dpo에 Trainer 등의 메타 데이터를 저장하고 이후 resume을 통해 추가 학습할 수 있도록 함.
adapter의 경우 /content/drive/Mydrive/멋사/adapters/에 저장
"""

In [1]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip install datasets peft trl bitsandbytes accelerate
!pip install -U transformers
!pip show transformers

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
print(os.getcwd())
print(os.listdir())

/content
['.config', '.env', 'drive', '.ipynb_checkpoints', 'sample_data']


In [5]:
!git clone https://github.com/jjjh02/AmoRe_crm_generator.git
%cd AmoRe_crm_generator
!git checkout jinhyeok
!git branch
os.chdir("/content/AmoRe_crm_generator")
print(os.getcwd())

Cloning into 'AmoRe_crm_generator'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 128 (delta 56), reused 99 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (128/128), 2.42 MiB | 7.05 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/AmoRe_crm_generator
Branch 'jinhyeok' set up to track remote branch 'jinhyeok' from 'origin'.
Switched to a new branch 'jinhyeok'
* jinhyeok
  main
/content/AmoRe_crm_generator


In [10]:
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from trl import DPOTrainer, DPOConfig

# 모델 및 경로 설정
MODEL_ID = "LGAI-EXAONE/EXAONE-4.0-1.2B"
CACHE_DIR = "./models"
OUTPUT_DIR = "./finetuning/checkpoints_dpo"
OUTPUT_ADAPTER_DIR = "/content/drive/MyDrive/멋사/adapters_dpo_new"
BASE_ADAPTER_PATH = "/content/drive/MyDrive/멋사/adapters_dpo/adapters_dpo_2"
NEW_ADAPTER_NAME = "dpo_adapter_v2"

# 데이터셋 경로 설정
DATA_DIR = "./finetuning/finetuning_data_dpo"
JSON_FILE = os.path.join(DATA_DIR, "cycle_01.json")

# 하이퍼파라미터 설정
PROMPT_LENGTH = 1024
MAX_SEQ_LENGTH = 1512


def load_dpo_dataset(json_path: str):
    """JSON 파일에서 DPO 형식의 데이터셋을 로드합니다.

    JSON 형식:
    [
      { "prompt": "...", "chosen": "...", "rejected": "..." },
      ...
    ]

    Args:
        json_path: JSON 파일 경로

    Returns:
        train_dataset, eval_dataset
    """
    # JSON 파일 로드
    dataset = load_dataset(
        "json",
        data_files=json_path,
    )
    dataset = dataset["train"]

    # train / eval split
    dataset = dataset.train_test_split(test_size=0.1, seed=42)

    return dataset["train"], dataset["test"]


def _freeze_all_params(model):
    for _, param in model.named_parameters():
        param.requires_grad = False


def _enable_adapter_params(model, adapter_name):
    for name, param in model.named_parameters():
        if f".{adapter_name}." in name:
            param.requires_grad = True


def main():
    ""DPO 학습 메인 함수""

    # 1. 토크나이저 로드
    print("토크나이저 로드 중...")
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_ID,
        cache_dir=CACHE_DIR,
    )

    # pad_token 설정
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # 패딩 사이드 설정 (DPO 학습에 유리)
    tokenizer.padding_side = 'left'
    tokenizer.truncation_side = 'left'

    # max_length 설정
    tokenizer.model_max_length = MAX_SEQ_LENGTH

    # 2. 데이터셋 로드
    print(f"데이터셋 로드 중: {JSON_FILE}")
    if not os.path.exists(JSON_FILE):
        raise FileNotFoundError(f"데이터셋 파일을 찾을 수 없습니다: {JSON_FILE}")

    train_dataset, eval_dataset = load_dpo_dataset(JSON_FILE)
    print(f"학습 데이터: {len(train_dataset)}개, 평가 데이터: {len(eval_dataset)}개")

    # 3. Flash Attention 설정
    if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8:
        attn_implementation = "flash_attention_2"
        torch_dtype = torch.bfloat16
    else:
        attn_implementation = "eager"
        torch_dtype = torch.float16

    # 4. 모델 로드
    print("모델 로드 중...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto"
        use_cache=False,
        attn_implementation=attn_implementation,
        torch_dtype=torch_dtype,
        cache_dir=CACHE_DIR,
    )

    # 5. PEFT (LoRA) 설정
    print("PEFT 설정 중...")
    peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05,
        r=64,
        bias="none"
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        task_type="CAUSAL_LM"
    )

    # 6. 베이스 어댑터 로드 (학습하지 않음)
    print(f"베이스 어댑터 로드 중: {BASE_ADAPTER_PATH}")
    if not os.path.exists(BASE_ADAPTER_PATH):
        raise FileNotFoundError(f"베이스 어댑터를 찾을 수 없습니다: {BASE_ADAPTER_PATH}")

    model = PeftModel.from_pretrained(
        model,
        BASE_ADAPTER_PATH,
        is_trainable=False,
    )

    # 7. 추가 어댑터 생성 및 활성화
    print(f"추가 어댑터 생성: {NEW_ADAPTER_NAME}")
    model.add_adapter(NEW_ADAPTER_NAME, peft_config)
    model.set_adapter(NEW_ADAPTER_NAME)
    _freeze_all_params(model)
    _enable_adapter_params(model, NEW_ADAPTER_NAME)

    # 8. DPO Config 설정
    print("DPO Config 설정 중...")
    dpo_config = DPOConfig(
        output_dir=OUTPUT_DIR,
        num_train_epochs=4,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=3,
        learning_rate=5e-5,
        max_grad_norm=0.3,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine"
        logging_steps=1,
        save_steps=100,
        save_total_limit=20,
        eval_strategy="steps"
        eval_steps=30000,
        fp16=True,
        beta=0.1,
        loss_type="sigmoid"
        report_to="none"
    )

    # 9. DPOTrainer 초기화
    print("DPOTrainer 초기화 중...")
    trainer = DPOTrainer(
        model=model,
        ref_model=None,  # PEFT 사용 시 None으로 설정
        args=dpo_config,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        processing_class=tokenizer,
    )

    # 10. 학습 시작
    print("학습 시작...")
    ckpt_dir = "AmoRe_crm_generator/finetuning/checkpoints_dpo"

    resume = None
    if os.path.isdir(ckpt_dir) and len(os.listdir(ckpt_dir)) > 0:
        resume = True

    trainer.train(resume_from_checkpoint=resume)

    # 11. 모델 저장
    print("모델 저장 중...")
    trainer.save_model(OUTPUT_ADAPTER_DIR)
    print(f"모델이 저장되었습니다: {OUTPUT_ADAPTER_DIR}")


if __name__ == "__main__":
    main()


토크나이저 로드 중...
데이터셋 로드 중: ./finetuning/finetuning_data_dpo/cycle_01.json
학습 데이터: 809개, 평가 데이터: 90개
BitsAndBytesConfig 설정 중...
모델 로드 중...
PEFT 설정 중...
DPO Config 설정 중...
DPOTrainer 초기화 중...


Extracting prompt in train dataset:   0%|          | 0/809 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/809 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/809 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/90 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


모델 저장 중...
모델이 저장되었습니다: ./finetuning/checkpoints_dpo


In [ ]:
!find . -name "cycle_01.csv"
!ls
!cd AmoRe_crm_generator && ls

AmoRe_crm_generator  data  models  README.md  requirements.txt	src
models
